In [ ]:
# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

In [104]:

# Задание 1.
# Обязательная часть
# Будем парсить страницу со свежеми новостям на habr.com/ru/all/.
# Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
# KEYWORDS = ['python', 'парсинг']
# Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
# В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>
# Дополнительная часть (необязательная)
# Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.
# Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
# Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

import requests
import time 
import pandas as pd
from bs4 import BeautifulSoup

# определяем список хабов, которые нам интересны
keywords = ['python', 'парсинг']
hub_news = pd.DataFrame()

# получаем страницу с самыми свежими постами
req = requests.get('http://habr.com/ru/all/')
soap = BeautifulSoup(req.text, 'html.parser')

# извлекаем посты
posts = soup.find_all('article', class_ ='post')

# Вариант 1

hubs = list(map(lambda x: x.find('a', class_='post__title_link'), posts))
hub_links = list(map(lambda x: x.find('a', class_='post__title_link').get('href'), posts))
hub_titles = list(map(lambda x: x.find('a', class_='post__title_link').getText(), posts))
hub_dates = list(map(lambda x: x.find('span', class_='post__time').getText(), posts))
hub_authors = list(map(lambda x: x.find('span', class_='user-info__nickname').getText(), posts))
hub_ids = list(map(lambda x: x.parent.attrs.get('id'), posts))

if len(hub_links) == len(hub_titles) == len(hub_dates) == len(hub_authors) == len(hub_ids):
    hub_news['hub_links']=hub_links
    hub_news['hub_titles']=hub_titles
    hub_news['hub_dates']=hub_dates
    hub_news['hub_authors']=hub_authors
    hub_news['hub_ids']=hub_ids
    hub_news = hub_news[hub_news['hub_titles'].apply(lambda x: any([keyword in x.lower() for keyword in keywords]))]
else:
    print ('Данный алгоритм не подходит')

# Вариант 2
hub_news2 = pd.DataFrame()

for index, post in enumerate(posts):
    # достаем id статьи
    hub_id = post.parent.attrs.get('id')
    # если идентификатор не найден, это что-то странное, пропускаем
    if not hub_id:
        continue
    # вычленяем числовую часть номера: делим 'post_127464' на части по разделителю '_' и забираем последнюю часть
    hub_id = int(hub_id.split('_')[-1]) 
    # достаем другие параметры статьи
    hub_link = post.find('a', class_='post__title_link').get('href')
    hub_title = post.find('a', class_='post__title_link').getText()
    hub_date = post.find('span', class_='post__time').getText()
    hub_author = post.find('span', class_='user-info__nickname').getText()
    hub_text = ''
    # проверяем заголовок на соответствие условиям поиска
    if any([keyword in hub_title.lower() for keyword in keywords]):
        # достаем полный текст статьи
        soup = BeautifulSoup(requests.get(hub_link).text, 'html.parser')
        time.sleep(0.2)
        if soap.find('div', class_='post__text post__text-html post__text_v1'):
            hub_text = soap.find('div', class_='post__text post__text-html post__text_v1').text
        # создаем итоговый датафрейм
        row = {'hub_title': hub_title, 'hub_date': hub_date, 'hub_author': hub_author, 'hub_link': hub_link, 'hub_id': hub_id, 'hub_text': hub_text}
        hub_news2 = hub_news2.append(pd.DataFrame([row]))    

hub_news2




""


In [141]:
# Задание 2. - 

# Обязательная часть
# Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
# EMAIL = [xxx@x.ru, yyy@y.com]
mail = {'emailAddresses': '["sas.coolteam@gmail.com"]'}
# В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

# Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

import requests
import time 
import pandas as pd
from bs4 import BeautifulSoup

req_url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Length': '45',
    'Content-Type': 'application/json;charset=UTF-8',
    'Host': 'identityprotection.avast.com',
    'Origin': 'https://www.avast.com',
    'Referer': 'https://www.avast.com/',
    'sec-ch-ua': '"Google Chrome";v="87", " Not;A Brand";v="99", "Chromium";v="87"',
    'sec-ch-ua-mobile': '?0',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Version': '0',
}

req = requests.post(req_url, json = mail, headers = headers)
req

# =======================================================
# Не получилось решить задачу - запрос возвращает пустоту
# =======================================================

time.sleep(0.3)
req_df = pd.DataFrame(req.json())



# {"breaches":{"132":{"breachId":132,"site":"vbulletin.com","recordsCount":522139,"description":"In March 2016, vBulletin's user database was allegedly breached. The breached data was first shared online in July 2016, by an unknown criminal, and has since been posted numerous times on various other underground forums.","publishDate":"2016-12-15T00:00:00Z","statistics":{"usernames":521945,"passwords":40707,"emails":522111}},"6":{"breachId":6,"site":"000webhost.com","recordsCount":15000649,"description":"An anonymous criminal was able to exploit an old, unpatched version of PHP to upload malicious files and gain access to 000webhost.com's systems. The full database containing usernames, passwords and email addresses was compromised and later dumped online. ","publishDate":"2016-10-24T00:00:00Z","statistics":{"usernames":14932947,"passwords":14998713,"emails":15000649}},"2":{"breachId":2,"site":"linkedin.com","recordsCount":158591429,"description":"In 2012, online professional networking platform LinkedIn fell victim to a breach of its members' passwords. Four years later, 117 million email and password combinations from that breach appeared for sale on a dark web marketplace. \n\nThe leaked passwords had only been protected by unsalted SHA-1 cryptographic hashing, which prompted LinkedIn to enforce salted hashing after the breach. Russian national Yevgeniy Nikulin was accused of the breach and was extradited from the Czech Republic to the United States as of March 2018.","publishDate":"2016-10-21T00:00:00Z","statistics":{"usernames":0,"passwords":111975337,"emails":158591429}},"16866":{"breachId":16866,"site":"firstvds.ru","recordsCount":125140,"description":"At an unconfirmed date, Russian server hosting service FirstVDS's database was allegedly breached. The stolen data contains usernames, passwords, email addresses and additional personal information. This breach is being privately shared on the internet.","publishDate":"2019-08-08T00:00:00Z","statistics":{"usernames":121480,"passwords":125051,"emails":122444}},"22":{"breachId":22,"site":"last.fm","recordsCount":39807558,"description":"In March 2012, the British music streaming service Last.fm suffered a major data breach when an unidentified criminal infiltrated and downloaded its user database. The incident was kept relatively quiet during the time of the attack until leaked data resurfaced in September 2016. Each leaked Last.fm record reportedly a username, email address, and password. Over 43 million users were affected. ","publishDate":"2016-10-25T00:00:00Z","statistics":{"usernames":39807459,"passwords":39807558,"emails":39310613}},"12":{"breachId":12,"site":"vk.com","recordsCount":91262655,"description":"Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised on a dark web marketplace as of June 2016 for a price of one bitcoin. ","publishDate":"2016-10-29T00:00:00Z","statistics":{"usernames":0,"passwords":89691243,"emails":91262653}},"3":{"breachId":3,"site":"adobe.com","recordsCount":152046506,"description":"In October of 2013, criminals penetrated Adobe's corporate network and the stole source code for several of its software products. The affected products included Adobe's ColdFusion web application platform as well as the Acrobat suite of products. Adobe asserts that criminals also accessed nearly three-million customer credit card records and stole login data for an undisclosed number of Adobe user accounts.","publishDate":"2016-10-21T00:00:00Z","statistics":{"usernames":0,"passwords":129430596,"emails":152046506}},"13324":{"breachId":13324,"site":"vbulletin.com","recordsCount":522320,"description":"In March 2016 vBulletin.com's database was breached. This stolen data contains usernames, emails, passwords and salts. This data is now publicly being shared on the darknet.","publishDate":"2018-01-07T00:00:00Z","statistics":{"usernames":522125,"passwords":522320,"emails":522219}},"26":{"breachId":26,"site":"hostinger.com","recordsCount":997176,"description":"In May 2015, Hostinger.com's database was breached by an unknown actor, and a little over a year later the username and encrypted password combinations from the database were privately shared online.","publishDate":"2016-10-23T00:00:00Z","statistics":{"usernames":992134,"passwords":996180,"emails":996991}},"16659":{"breachId":16659,"site":"livejournal.com","recordsCount":30485836,"description":"In 2017, social network LiveJournal's database was allegedly breached. The stolen data contains passwords, email addresses and additional personal information. This breach is being privately shared in the dark web.","publishDate":"2019-05-23T00:00:00Z","statistics":{"usernames":30485732,"passwords":29634403,"emails":30482018}}},"data":{"000webhost.com":{"sas.coolteam@gmail.com":[{"breachId":6,"usernameBreached":true,"passwordBreached":true}]},"last.fm":{"sas.coolteam@gmail.com":[{"breachId":22,"usernameBreached":true,"passwordBreached":true}]},"hostinger.com":{"sas.coolteam@gmail.com":[{"breachId":26,"usernameBreached":true,"passwordBreached":true}]},"adobe.com":{"sas.coolteam@gmail.com":[{"breachId":3,"usernameBreached":true,"passwordBreached":true}]},"linkedin.com":{"sas.coolteam@gmail.com":[{"breachId":2,"usernameBreached":true,"passwordBreached":true}]},"livejournal.com":{"sas.coolteam@gmail.com":[{"breachId":16659,"usernameBreached":true,"passwordBreached":true}]},"vbulletin.com":{"sas.coolteam@gmail.com":[{"breachId":132,"usernameBreached":true,"passwordBreached":true},{"breachId":13324,"usernameBreached":true,"passwordBreached":true}]},"vk.com":{"sas.coolteam@gmail.com":[{"breachId":12,"usernameBreached":true,"passwordBreached":true}]},"firstvds.ru":{"sas.coolteam@gmail.com":[{"breachId":16866,"usernameBreached":true,"passwordBreached":true}]}},"summary":{"sas.coolteam@gmail.com":{"breaches":[132,6,2,16866,22,12,3,13324,26,16659]}}}


NameError: name 'true' is not defined

In [ ]:
# Дополнительная часть (необязательная)
# Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
# Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

# GROUP = 'netology'  
# TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
# В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>


# params = {
#     'target_type': 'posts',
#     'q': {keywords[0]},
#     'order_by': 'date'
# }
# URL = 'https://habr.com/ru/search/'
# headers = {}

# time.sleep(0.2)

# req = requests.get(URL, params)
# req = requests.post(URL, params, headers)
# req.text

# soup = BeautifulSoup(req.text, 'html.parser')
# posts = soup.find_all('article', class_='post')

# for post in posts:
#     post_id = post.parent.attrs.get('id')
#     if not post_id:
#         continue
#     post_id = int(post_id)